In [1]:
from pyspark.sql import SparkSession

import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.driver.memory=8G pyspark-shell'

spark = (SparkSession
         .builder
         .master("local[*]")
         .getOrCreate())

In [24]:
# the first step is to read in the files

initialData = sc.textFile("web-Google.txt")
initialData = initialData.map(lambda x: (x.split("\t")[0], x.split("\t")[1]))
print(initialData.collect())


[('0', '11342'), ('0', '824020'), ('0', '867923'), ('0', '891835'), ('11342', '0'), ('11342', '27469'), ('11342', '38716'), ('11342', '309564'), ('11342', '322178'), ('11342', '387543'), ('11342', '427436'), ('11342', '538214'), ('11342', '638706'), ('11342', '645018'), ('11342', '835220'), ('11342', '856657'), ('11342', '867923'), ('11342', '891835'), ('824020', '0'), ('824020', '91807'), ('824020', '322178'), ('824020', '387543'), ('824020', '417728'), ('824020', '438493'), ('824020', '500627'), ('824020', '535748'), ('824020', '695578'), ('824020', '867923'), ('824020', '891835'), ('867923', '0'), ('867923', '11342'), ('867923', '136593'), ('867923', '414038'), ('867923', '500627'), ('867923', '523684'), ('867923', '760842'), ('867923', '815602'), ('867923', '835220'), ('867923', '846213'), ('867923', '857527'), ('867923', '891835'), ('891835', '0'), ('891835', '11342'), ('891835', '112028'), ('891835', '235849'), ('891835', '302284'), ('891835', '417728'), ('891835', '451592'), ('8

In [28]:
#second step is to create a links rdd with all the outgoing links from a page
links = initialData.reduceByKey(lambda x,y: str(x) + " "+ str(y))
print(links.collect())

[('0', '11342 824020 867923 891835'), ('867923', '0 11342 136593 414038 500627 523684 760842 815602 835220 846213 857527 891835'), ('891835', '0 11342 112028 235849 302284 417728 451592 693969 857527 867923'), ('1', '53051 203402 223236 276233 552600 569212 635575 748615 862566 893884'), ('276233', '1 53051 164684 203402 208814 223236 240041 303045 363925 384341 408707 436135 552600 569212 619841 635575 748615 773119 774614 792266 838771 862566 893884'), ('569212', '1 53051 164684 184896 184984 203402 208814 223236 240041 276233 284930 363925 436135 449352 539143 552600 573555 604413 635575 648208 748615 774614 789543 862566 893884'), ('748615', '1 53051 164684 184896 184984 203402 208814 223236 240041 276233 284930 303045 363925 384341 408707 436135 449352 532018 539143 552600 569212 573555 604413 619841 635575 648208 773119 774614 789543 792151 792266 824625 838771 862566 893884'), ('862566', '1 53051 164684 184896 184984 203402 208814 223236 240041 276233 284930 303045 436135 449352

In [33]:
#third step is to initialize a ranks rdd with all the ranks and the ranks it will give to 
#the outgoing links
ranks = links.map(lambda x: (x, 1)).map(lambda x: (x[0][0], x[0][1],1, 1 / len(x[0][1].split(" ")))) 
print(ranks.collect())

[('0', '11342 824020 867923 891835', 1, 0.25), ('867923', '0 11342 136593 414038 500627 523684 760842 815602 835220 846213 857527 891835', 1, 0.08333333333333333), ('891835', '0 11342 112028 235849 302284 417728 451592 693969 857527 867923', 1, 0.1), ('1', '53051 203402 223236 276233 552600 569212 635575 748615 862566 893884', 1, 0.1), ('276233', '1 53051 164684 203402 208814 223236 240041 303045 363925 384341 408707 436135 552600 569212 619841 635575 748615 773119 774614 792266 838771 862566 893884', 1, 0.043478260869565216), ('569212', '1 53051 164684 184896 184984 203402 208814 223236 240041 276233 284930 363925 436135 449352 539143 552600 573555 604413 635575 648208 748615 774614 789543 862566 893884', 1, 0.04), ('748615', '1 53051 164684 184896 184984 203402 208814 223236 240041 276233 284930 303045 363925 384341 408707 436135 449352 532018 539143 552600 569212 573555 604413 619841 635575 648208 773119 774614 789543 792151 792266 824625 838771 862566 893884', 1, 0.0285714285714285

In [2]:
def define(arr, rank):
    temp = []
    arrTemp = arr.split(" ")
    for num in arrTemp:
        temp.append((num, rank))
    return temp

In [44]:
#producing the different ranks each page would get
trial = ranks.flatMap(lambda x: define(x[1],x[3]))
print(trial.collect())

[('11342', 0.25), ('824020', 0.25), ('867923', 0.25), ('891835', 0.25), ('0', 0.08333333333333333), ('11342', 0.08333333333333333), ('136593', 0.08333333333333333), ('414038', 0.08333333333333333), ('500627', 0.08333333333333333), ('523684', 0.08333333333333333), ('760842', 0.08333333333333333), ('815602', 0.08333333333333333), ('835220', 0.08333333333333333), ('846213', 0.08333333333333333), ('857527', 0.08333333333333333), ('891835', 0.08333333333333333), ('0', 0.1), ('11342', 0.1), ('112028', 0.1), ('235849', 0.1), ('302284', 0.1), ('417728', 0.1), ('451592', 0.1), ('693969', 0.1), ('857527', 0.1), ('867923', 0.1), ('53051', 0.1), ('203402', 0.1), ('223236', 0.1), ('276233', 0.1), ('552600', 0.1), ('569212', 0.1), ('635575', 0.1), ('748615', 0.1), ('862566', 0.1), ('893884', 0.1), ('1', 0.043478260869565216), ('53051', 0.043478260869565216), ('164684', 0.043478260869565216), ('203402', 0.043478260869565216), ('208814', 0.043478260869565216), ('223236', 0.043478260869565216), ('24004

In [51]:
#adding it all up and introducing the damping constant
rankedSum = trial.reduceByKey(lambda x,y: x + y).map(lambda x: (x[0], x[1] * 0.85)).map(lambda x: (x[0], x[1] + 0.15) )
print(rankedSum.collect())

[('867923', 0.585487012987013), ('891835', 0.5713203463203463), ('0', 0.4438203463203463), ('136593', 0.22083333333333333), ('523684', 0.22083333333333333), ('815602', 0.22083333333333333), ('835220', 0.28154761904761905), ('857527', 0.30583333333333335), ('112028', 0.235), ('235849', 0.235), ('302284', 0.235), ('417728', 0.43032828282828284), ('693969', 0.235), ('276233', 0.4860767195767196), ('569212', 0.48903324131585), ('748615', 0.4987475270301357), ('862566', 0.4915517598343685), ('893884', 0.48053324131584996), ('1', 0.35505705083965955), ('164684', 0.3050570508396595), ('303045', 0.27105705083965953), ('384341', 0.23957556935817803), ('408707', 0.23957556935817803), ('436135', 0.35505705083965955), ('774614', 0.32672371750632623), ('184896', 0.3106005291005291), ('184984', 0.3106005291005291), ('284930', 0.3106005291005291), ('449352', 0.3106005291005291), ('539143', 0.3106005291005291), ('532018', 0.20261904761904762), ('824625', 0.25261904761904763), ('430119', 0.835188422688

In [59]:
#it is time to update the ranks
combine = ranks.join(rankedSum).sortBy(lambda a: a[0], ascending = True)
print(combine.collect())

[('0', ('11342 824020 867923 891835', 0.4438203463203463)), ('1', ('53051 203402 223236 276233 552600 569212 635575 748615 862566 893884', 0.35505705083965955)), ('10', ('49988 85506 129774 134964 181848 194746 310676 332640 393685 556796 826668', 1.8074999999999999)), ('101', ('132198 150432 242339 297892 331855 482167 505991 676971 677084 901704', 0.235)), ('101006', ('147162 262316 325667 432149 448711 517626 627839 799322 859399', 0.7764116161616161)), ('101821', ('108 98207 303790 540930 574730 596374', 0.9230952380952382)), ('102', ('493731 590960 735590 769753', 1.0)), ('102352', ('133304 345509 411227 452525 580836 671110 854120', 1.2167216810966808)), ('102669', ('85 17093 29338 43515 80527 105956 109485 130784 264427 285623 302064 438505 444796 467572 492321 864802 887810 906432', 0.5167593631631102)), ('103', ('142218 336700 623548 720240 821098 840920', 0.25625)), ('103454', ('60543 133420 331197 436950 500896 772466', 0.2153846153846154)), ('104279', ('143673 257316 298314

In [60]:
combine = combine.map(lambda x: (x[0], x[1][0], 1 , x[1][1]))
print(combine.collect())

[('0', '11342 824020 867923 891835', 1, 0.4438203463203463), ('1', '53051 203402 223236 276233 552600 569212 635575 748615 862566 893884', 1, 0.35505705083965955), ('10', '49988 85506 129774 134964 181848 194746 310676 332640 393685 556796 826668', 1, 1.8074999999999999), ('101', '132198 150432 242339 297892 331855 482167 505991 676971 677084 901704', 1, 0.235), ('101006', '147162 262316 325667 432149 448711 517626 627839 799322 859399', 1, 0.7764116161616161), ('101821', '108 98207 303790 540930 574730 596374', 1, 0.9230952380952382), ('102', '493731 590960 735590 769753', 1, 1.0), ('102352', '133304 345509 411227 452525 580836 671110 854120', 1, 1.2167216810966808), ('102669', '85 17093 29338 43515 80527 105956 109485 130784 264427 285623 302064 438505 444796 467572 492321 864802 887810 906432', 1, 0.5167593631631102), ('103', '142218 336700 623548 720240 821098 840920', 1, 0.25625), ('103454', '60543 133420 331197 436950 500896 772466', 1, 0.2153846153846154), ('104279', '143673 257

In [61]:
combine = combine.map(lambda x: (x[0], x[1], x[2] , x[3] / len(x[1].split(" "))))
print(combine.collect())

[('0', '11342 824020 867923 891835', 1, 0.11095508658008657), ('1', '53051 203402 223236 276233 552600 569212 635575 748615 862566 893884', 1, 0.03550570508396596), ('10', '49988 85506 129774 134964 181848 194746 310676 332640 393685 556796 826668', 1, 0.1643181818181818), ('101', '132198 150432 242339 297892 331855 482167 505991 676971 677084 901704', 1, 0.0235), ('101006', '147162 262316 325667 432149 448711 517626 627839 799322 859399', 1, 0.08626795735129068), ('101821', '108 98207 303790 540930 574730 596374', 1, 0.15384920634920637), ('102', '493731 590960 735590 769753', 1, 0.25), ('102352', '133304 345509 411227 452525 580836 671110 854120', 1, 0.17381738301381153), ('102669', '85 17093 29338 43515 80527 105956 109485 130784 264427 285623 302064 438505 444796 467572 492321 864802 887810 906432', 1, 0.028708853509061678), ('103', '142218 336700 623548 720240 821098 840920', 1, 0.04270833333333333), ('103454', '60543 133420 331197 436950 500896 772466', 1, 0.0358974358974359), ('

In [62]:
t = ranks.sortBy(lambda a: a[0], ascending = True)
print(t.collect())

[('0', '11342 824020 867923 891835', 1, 0.25), ('1', '53051 203402 223236 276233 552600 569212 635575 748615 862566 893884', 1, 0.1), ('10', '49988 85506 129774 134964 181848 194746 310676 332640 393685 556796 826668', 1, 0.09090909090909091), ('101', '132198 150432 242339 297892 331855 482167 505991 676971 677084 901704', 1, 0.1), ('101006', '147162 262316 325667 432149 448711 517626 627839 799322 859399', 1, 0.1111111111111111), ('101821', '108 98207 303790 540930 574730 596374', 1, 0.16666666666666666), ('102', '493731 590960 735590 769753', 1, 0.25), ('102352', '133304 345509 411227 452525 580836 671110 854120', 1, 0.14285714285714285), ('102669', '85 17093 29338 43515 80527 105956 109485 130784 264427 285623 302064 438505 444796 467572 492321 864802 887810 906432', 1, 0.05555555555555555), ('103', '142218 336700 623548 720240 821098 840920', 1, 0.16666666666666666), ('103454', '60543 133420 331197 436950 500896 772466', 1, 0.16666666666666666), ('104279', '143673 257316 298314 317

In [3]:
import numpy as np

In [4]:
#this was the basic one iteration of the PageRank. Now we can do it for an x amount of 
#iterations with constant updates

#rdd here is the file which has just been created by reading the file
def PageRank(rdd, iterations):
    links = rdd.reduceByKey(lambda x,y: str(x) + " "+ str(y))
    ranks = links.map(lambda x: (x, 1)) \
                 .map(lambda x: (x[0][0], x[0][1],1, 1 / len(x[0][1].split(" ")))) 
    for i in np.arange(0, iterations):
        trial = ranks.flatMap(lambda x: define(x[1],x[3]))
        rankedSum = trial.reduceByKey(lambda x,y: x + y) \
                         .map(lambda x: (x[0], x[1] * 0.85)) \
                         .map(lambda x: (x[0], x[1] + 0.15))
                              
        combined = ranks.join(rankedSum) \
                        .sortBy(lambda a: a[0], ascending = True) \
                        .map(lambda x: (x[0], x[1][0], 1 , x[1][1]))

        ranks = combined.map(lambda x: (x[0], x[1], x[2] , x[3] / len(x[1].split(" "))))
    return ranks                                                                                            
        

In [5]:
initial = sc.textFile("web-Google.txt")
initial = initial.map(lambda x: (x.split("\t")[0], x.split("\t")[1]))

In [6]:
tri = PageRank(initial, 10)

In [7]:
tri.collect()

[('0', '11342 824020 867923 891835', 1, 0.05437592117954571),
 ('1',
  '53051 203402 223236 276233 552600 569212 635575 748615 862566 893884',
  1,
  0.019565735705929403),
 ('10',
  '49988 85506 129774 134964 181848 194746 310676 332640 393685 556796 826668',
  1,
  0.14323706465566843),
 ('101',
  '132198 150432 242339 297892 331855 482167 505991 676971 677084 901704',
  1,
  0.02041092851871896),
 ('101006',
  '147162 262316 325667 432149 448711 517626 627839 799322 859399',
  1,
  0.03141813736575194),
 ('101821', '108 98207 303790 540930 574730 596374', 1, 0.1095388707928824),
 ('102', '493731 590960 735590 769753', 1, 0.25),
 ('102352',
  '133304 345509 411227 452525 580836 671110 854120',
  1,
  0.11677859812097206),
 ('102669',
  '85 17093 29338 43515 80527 105956 109485 130784 264427 285623 302064 438505 444796 467572 492321 864802 887810 906432',
  1,
  0.013938838228583779),
 ('103', '142218 336700 623548 720240 821098 840920', 1, 0.029256720913777404),
 ('104279',
  '143673